In [16]:
from pdf2image import convert_from_path
import cloudinary
import cloudinary.uploader
import os

def pdf_to_images_and_upload(pdf_path):
    # Cấu hình Cloudinary bằng các biến môi trường
    cloudinary.config(
        cloud_name="dfatgrcng",
        api_key="835866625462259",
        api_secret="qMX2WoR9nn7OWmGVvbK5aYRwP8o"
    )

    # Chuyển đổi pdf thành hình ảnh
    images = convert_from_path(pdf_path)
    image_links = []
    for page_num, image in enumerate(images):
        image_filename = f"page_{page_num + 1}.png"
        image.save(image_filename, 'JPEG')
        # Tải hình ảnh lên Cloudinary
        result = cloudinary.uploader.upload(image_filename)
        image_links.append(result['secure_url'])
        # Xóa hình ảnh sau khi tải lên
        os.remove(image_filename)
    return image_links

In [25]:
pdf_to_images_and_upload("C:/Users/PC/Desktop/Slide/data/data/999.pdf")

['https://res.cloudinary.com/dfatgrcng/image/upload/v1727839607/iznbloonilyicgfb9tkr.jpg',
 'https://res.cloudinary.com/dfatgrcng/image/upload/v1727839608/jnglhanjgs3ly72evoii.jpg',
 'https://res.cloudinary.com/dfatgrcng/image/upload/v1727839609/n4b21bfgfzh1tfmixzno.jpg',
 'https://res.cloudinary.com/dfatgrcng/image/upload/v1727839610/niajrylre6islgh93hmg.jpg',
 'https://res.cloudinary.com/dfatgrcng/image/upload/v1727839611/xmulbywmyq2zd8mgyv9z.jpg',
 'https://res.cloudinary.com/dfatgrcng/image/upload/v1727839613/vbxt2mg04s1strxgx2og.jpg',
 'https://res.cloudinary.com/dfatgrcng/image/upload/v1727839614/rcxnkluqnsxgnzphzm8s.jpg',
 'https://res.cloudinary.com/dfatgrcng/image/upload/v1727839616/a32xyjzg2cuxrfs9snzh.jpg',
 'https://res.cloudinary.com/dfatgrcng/image/upload/v1727839618/pgcflp575ppijn3x6sfy.jpg',
 'https://res.cloudinary.com/dfatgrcng/image/upload/v1727839619/qbyiliunglsd1yzukrwy.jpg',
 'https://res.cloudinary.com/dfatgrcng/image/upload/v1727839621/kledtq0uyd7znrob42zz.jpg',

In [40]:
import os
from together import Together
client = Together(api_key='28fe2cf1289992f74fff42816b194e166513bf2d8dce2c0228dce23527d0df65')
def extract_text_with_prompt_from_image(image_url,prompt):
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.2-90B-Vision-Instruct-Turbo",
        messages=[
            {
                    "role": "user",
                    "content": [
                            {
                                    "type": "text",
                                    "text": prompt if prompt else "Extract the text from the image"
                            },
                            {
                                    "type": "image_url",
                                    "image_url": {
                                            "url": image_url
                                    }
                            }
                    ]
            }
    ],
        max_tokens=512,
        temperature=0.7,
        top_p=0.7,
        top_k=50,
        repetition_penalty=1,
        stop=["<|eot_id|>","<|eom_id|>"],
    )
    # print('response',response)
    return response.choices[0].message.content if response.choices else "Unable to process the image with the provided prompt."

In [38]:
# import os
# from together import Together

# client = Together(api_key='28fe2cf1289992f74fff42816b194e166513bf2d8dce2c0228dce23527d0df65')

# def extract_text_with_prompt_from_image(image_url, prompt):
#     # Combine prompt and image URL into a single string content
#     content = f"{prompt if prompt else 'Extract the text from the image'}\nImage URL: {image_url}"

#     response = client.chat.completions.create(
#         model="meta-llama/Llama-3.2-90B-Vision-Instruct-Turbo",
#         messages=[
#             {
#                 "role": "user",
#                 "content": content  # Pass a single string
#             }
#         ],
#         max_tokens=512,
#         temperature=0.7,
#         top_p=0.7,
#         top_k=50,
#         repetition_penalty=1,
#         stop=["<|eot_id|>", "<|eom_id|>"],
#     )
    
#     # print('response',response)
#     return response.choices[0].message.content if response.choices else "Unable to process the image with the provided prompt."


In [42]:
prompt = """Trích xuất văn bản từ hình ảnh. Chỉ trả về văn bản, không cần giải thích."""
extract_text_with_prompt_from_image("https://res.cloudinary.com/dfatgrcng/image/upload/v1727834913/qq6xoywutdsivoa2m6dm.jpg",prompt)

ValidationError: 1 validation error for ChatCompletionRequest
messages.0.content
  Input should be a valid string [type=string_type, input_value=[{'type': 'text', 'text':...ywutdsivoa2m6dm.jpg']}}], input_type=list]
    For further information visit https://errors.pydantic.dev/2.8/v/string_type

In [22]:
from PyPDF2 import PdfReader

def get_pdf_text(pdf_docs):

    text = ""

    # Check if pdf_docs is a list or a string
    if not isinstance(pdf_docs, list):
        pdf_docs = [pdf_docs] # Convert to a list if it's a string

    # Thử đọc từng file PDF trong danh sách
    for pdf in pdf_docs:
        try:
            pdf_reader = PdfReader(pdf)  # pdf là đường dẫn của từng file PDF
            for page in pdf_reader.pages:
                text += page.extract_text() or ""
        except IsADirectoryError: # Handle the case where the input is a directory
            print(f"Error: {pdf} is a directory, not a PDF file.")
        except FileNotFoundError: # Handle the case where the file doesn't exist
            print(f"Error: {pdf} not found.")
        except PyPDF2.errors.PdfReadError: # Handle the case where the file is not a valid PDF
            print(f"Error: {pdf} is not a valid PDF file.")

        # Nếu văn bản trả về rỗng, chuyển file PDF thành ảnh và trích xuất văn bản từ ảnh
        if not text.strip():
            image_urls = pdf_to_images_and_upload(pdf)  # Chuyển đổi từng file PDF thành ảnh
            prompt = """Trích xuất văn bản từ hình ảnh. Chỉ trả về văn bản, không cần giải thích."""
            for image_url in image_urls:
                response = extract_text_with_prompt_from_image(image_url, prompt)
                text += response.strip() + "\n\n"
    
    return text

In [23]:
get_pdf_text(["C:/Users/PC/Desktop/Slide/data/data/999.pdf"])

ValidationError: 1 validation error for ChatCompletionRequest
messages.0.content
  Input should be a valid string [type=string_type, input_value=[{'type': 'text', 'text':...8e99fns1jk8egopq.jpg'}}], input_type=list]
    For further information visit https://errors.pydantic.dev/2.8/v/string_type